### Half Moons

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

data, labels = datasets.make_moons(n_samples=1000, shuffle=True, noise=0.2, random_state=None)
plt.scatter(data.T[0],data.T[1])

In [ ]:
# Divide training test
from sklearn import model_selection
data_train, data_test, labels_train, labels_test =  model_selection.train_test_split(data, labels, test_size=.5, train_size=.5, random_state=None, shuffle=True, stratify=None)
plt.scatter(data_train.T[0], data_train.T[1], c = labels_train)

In [ ]:
# Growing Spheres:
from scipy.special import gammainc
from matplotlib import pyplot as plt


def GrowingSpheres(clf, observation, m, n, dimensions):
    stop = False
    x_pred = clf.predict([observation]) # in {-1, 1}
    while not stop:
        z = GenerateOnSL(observation, 0, m, n, dimensions)
        z_pred = clf.predict(z)
        stop = all(x_pred == z_unit for z_unit in z_pred)
        m = m/2
    a_0 = m
    a_1 = 2*m
    stop = False
    while all(z_unit == x_pred for z_unit in z_pred):
        z = GenerateOnSL(observation, a_0, a_1, n, dimensions)
        z_pred = clf.predict(z)
        a_0 = a_1
        a_1 = a_1 + m
    norm_distances = [np.linalg.norm(observation -z[i]) if z_pred[i] != x_pred else 0 for i in range(len(z_pred))]
    #for i, z_unit in enumerate(z):
    #    if(z_predz)
    return z[np.argmax(norm_distances)]
    
def GenerateOnSL(center, minR, maxR, nbSamples, dimensions):
    # we obtain r varying between minR and maxR
    R = np.random.uniform(minR, maxR, size = nbSamples)
    points = []
    x = np.random.normal(scale=1,size=(nbSamples, dimensions))
    x_norm = [x[i]/ np.linalg.norm(x, axis = 1)[i] for i in range(x.shape[0])]
    for i, x_point in enumerate(x_norm):
        r = np.random.uniform(minR,maxR)
        points.append(x_point * R[i] + center )
    return np.array(points)

In [ ]:
from cProfile import label
from mlxtend.plotting import plot_decision_regions

def plotEverything(X,Y,classifier,title, obs, enemy):


    plot_decision_regions(X, Y, clf=classifier, legend=2)
    
    plt.scatter(obs[0], obs[1], c = 'lime',marker= 'x')
    plt.scatter(enemy.T[0], enemy.T[1], c = 'red', marker = 'o')
    # Adding axes annotations
    plt.title(title)
    plt.show()

In [ ]:
choice = np.random.choice(len(data_test))
#obs = [data_test[choice]]
obs = [2,0]

In [ ]:

# SVM
#choice = np.random.choice(len(data_test))
#obs = data_test[choice]
from sklearn import svm

svmClassifier = svm.SVC()
svmClassifier.fit(data_train, labels_train)
prediction = svmClassifier.predict(data_test)
enemy = GrowingSpheres(svmClassifier, obs, 0.1, 20,2)
plotEverything(data_test,prediction,svmClassifier,'SVM', obs, enemy)

In [ ]:
# KNNN
from sklearn import neighbors
#choice = np.random.choice(len(data_test))
#obs = data_test[choice]

knnClassifier = neighbors.KNeighborsClassifier(n_neighbors=3)
knnClassifier.fit(data_train, labels_train)
prediction = knnClassifier.predict(data_test)
enemy = GrowingSpheres(knnClassifier, obs, 0.1, 20,2)
plotEverything(data_test,prediction,knnClassifier,'K-neighbors', obs, enemy)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

#choice = np.random.choice(len(data_test))
#obs = data_test[choice]

rfClassifier = RandomForestClassifier(max_depth=4, random_state=0)
rfClassifier.fit(data_train, labels_train)
prediction = rfClassifier.predict(data_test)
enemy = GrowingSpheres(rfClassifier, obs, 0.1, 20,2)
plotEverything(data_test,prediction,rfClassifier,'Random Forest', obs, enemy)

In [ ]:
# SVM varier n
#choice = np.random.choice(len(data_test))
#obs = data_test[choice]
from sklearn import svm

svmClassifier = svm.SVC()
svmClassifier.fit(data_train, labels_train)
prediction = svmClassifier.predict(data_test)
n_values = np.linspace(2, 50, num=10, dtype=int)
for n in n_values:
    enemy = GrowingSpheres(svmClassifier, obs, 0.1, n, 2)
    plotEverything(data_test,prediction,svmClassifier,'SVM n:'+str(n), obs, enemy)

In [ ]:
# SVM varier m
#choice = np.random.choice(len(data_test))
#obs = data_test[choice]

svmClassifier = svm.SVC()
svmClassifier.fit(data_train, labels_train)
prediction = svmClassifier.predict(data_test)
m_values = np.linspace(0.1,2, num=10)
for m in m_values:
    enemy = GrowingSpheres(svmClassifier, obs, m, 20, 2)
    plotEverything(data_test,prediction,svmClassifier,'SVM m:'+str(m), obs, enemy)

### Breast Cancer Winsconsin

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
breast_cancer = datasets.load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(breast_cancer['data'], breast_cancer['target'], train_size=0.75, random_state = 1234)


In [ ]:

obs = X_test[0]
svmClassifier = svm.SVC()
svmClassifier.fit(X_train, y_train)
prediction = svmClassifier.predict(X_test)


In [ ]:
from tqdm import tqdm
dist = np.zeros_like(obs)
best_norm = np.Inf
for i in tqdm(range(20)):
    enemy = GrowingSpheres(svmClassifier, obs, 0.5, 50, 30)
    dist += obs - enemy
    norm = np.linalg.norm(obs - enemy)
    if norm < best_norm:
        best_norm = norm
        best_enemy = enemy
dist = dist / 20

In [ ]:
print(dist)
obsprediction = svmClassifier.predict(obs.reshape(1, -1))
enemyprediction = svmClassifier.predict(best_enemy.reshape(1, -1))
feature_names = breast_cancer['feature_names']
target_names = breast_cancer['target_names']
print(feature_names)
print(target_names)

ind = np.arange(len(obs))


def ylabeldata(ind, data, labels, title):
    fig, ax = plt.subplots()
    p1 = ax.barh(ind, data, label='Men')
    ax.set_title(title)
    ax.set_yticks(ind, labels=labels)

In [ ]:

ylabeldata(ind, obs, feature_names, "Observation, class = " + str(target_names[obsprediction]))

In [ ]:
dist1 = obs - best_enemy
ylabeldata(ind, np.abs(dist1), feature_names, "Distance conterfactual - observation")

In [ ]:
ylabeldata(ind,  np.abs(dist), feature_names, "Mean distances (all CF)")

In [ ]:
ylabeldata(ind, best_enemy, feature_names, "best conterfactual, class = " + str(target_names[enemyprediction]))